# ERC-4626: vaults across all chains

This is a [research notebook](https://tradingstrategy.ai/glossary/jupyter-notebook) for listing and analysing ERC-4626 vaults.

- In this notebook, we examine different [ERC-4626 vaults](https://tradingstrategy.ai/glossary/erc-4626) across different [EVM blockchains](https://tradingstrategy.ai/glossary/evm-compatible)
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- First you need the data. See *ERC-4626: scanning vault data* tutorial to get the data locally.
- Then just run this notebook in Visual Studio Code.

## Setup

- Set up notebook renderinb parmaeters

In [1]:
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [2]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 10907 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [3]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like
from eth_defi.research.vault_metrics import format_vault_database

df = format_vault_database(vault_db)

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())

DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Protocol, Denomination, NAV, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


,,Symbol,Name,Protocol,Denomination,NAV,Mgmt fee,Perf fee,Shares,First seen,_detection_data,...,_share_token,Protocol identified,Stablecoin denominated,ERC-7540,ERC-7575,Fee detected,Deposit count,Redeem count,Total events,Age
Chain,Address,,,,,,,,,,,,,,,,,,,,,
Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,BBiBGT,Beraborrow iBGT,<unknown ERC-4626>,iBGT,"139,458.31",<unknown>,<unknown>,73733.540231643576655477,2025-03-07 12:05:19,"ERC4262VaultDetection(chain=80094, address='0x...",...,"{'name': 'Beraborrow iBGT', 'symbol': 'BBiBGT'...",False,False,False,False,False,705249,656250,1361499,159 days 02:59:06.009241
Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,sDAI,Savings xDAI,<unknown ERC-4626>,WXDAI,"109,877,625.31",<unknown>,<unknown>,92066122.682571977791686786,2023-09-28 12:13:30,"ERC4262VaultDetection(chain=100, address='0xaf...",...,"{'name': 'Savings xDAI', 'symbol': 'sDAI', 'to...",False,True,False,False,False,478528,441846,920374,685 days 02:50:55.009241
Berachain,0x69f1e971257419b1e9c405a553f252c64a29a30a,oriBGT,Origami iBGT Auto-Compounder,<unknown ERC-4626>,iBGT,"5,000,852.20",<unknown>,<unknown>,2625027.971747689830298582,2025-03-17 01:22:21,"ERC4262VaultDetection(chain=80094, address='0x...",...,"{'name': 'Origami iBGT Auto-Compounder', 'symb...",False,False,False,False,False,640763,180234,820997,149 days 13:42:04.009241
Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,UB-WETH,UB-WETH,<unknown ERC-4626>,WETH,44.54,<unknown>,<unknown>,44.330727101729306253,2025-01-13 06:53:05,"ERC4262VaultDetection(chain=1868, address='0x2...",...,"{'name': 'UB-WETH', 'symbol': 'UB-WETH', 'tota...",False,False,False,False,False,590503,205871,796374,212 days 08:11:20.009241
Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,yvUSDC-A,USDC yVault-A,Yearn v3,USDC,"209,509.35",<unknown>,<unknown>,175544.387574,2023-10-26 16:45:48,"ERC4262VaultDetection(chain=137, address='0xa0...",...,"{'name': 'USDC yVault-A', 'symbol': 'yvUSDC-A'...",True,True,False,False,False,465226,25374,490600,656 days 22:18:37.009241


## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [4]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby(level="Chain").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby(level="Chain")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby(level="Chain").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby(level="Chain").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold:,} USD)": df[notable_nav].groupby(level="Chain").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby(level="Chain").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby(level="Chain").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby(level="Chain").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,"Notable stablecoin NAV (min 100,000 USD)",ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,2178,"225,243,841.75",405.00,751,70.00,36.00,44.00,253.00
Avalanche,376,"339,603,121.72",68.00,113,23.00,1.00,1.00,48.00
Base,1964,"990,169,948.27",833.00,943,87.00,49.00,28.00,157.00
Berachain,749,"56,665,053.17",64.00,30,11.00,0.00,0.00,7.00
Binance,418,"388,042,573.27",65.00,125,26.00,1.00,1.00,28.00
Blast,107,"842,623.21",2.00,14,2.00,0.00,0.00,32.00
Celo,10,"3,221,597.26",0.00,7,1.00,0.00,0.00,0.00
Ethereum,2756,"16,463,033,836.99",814.00,1005,298.00,35.00,33.00,279.00
Gnosis,93,"112,301,360.60",13.00,39,5.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [5]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby("Protocol").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby("Protocol")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby("Protocol").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby("Protocol").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold} USD)": df[notable_nav].groupby("Protocol").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby("Protocol").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby("Protocol").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby("Protocol").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,7772,"13,196,991,396.51",0.00,"2,780.00",309.00,0.00,12.00,0.00
Morpho,300,"2,603,173,999.84",300.00,183.00,89.00,0.00,0.00,300.00
<unknown ERC-7540>,51,"946,654,157.27",0.00,28.00,6.00,51.00,47.00,0.00
Euler Vault Kit,514,"636,011,576.78",514.00,132.00,48.00,0.00,0.00,0.00
Fluid,67,"622,732,412.12",67.00,28.00,9.00,0.00,0.00,0.00
Kiln Metavault,106,"345,127,180.58",106.00,90.00,36.00,0.00,0.00,0.00
Yearn v3,182,"253,965,661.05",182.00,90.00,23.00,0.00,0.00,0.00
Peapods,190,"31,087,731.05",190.00,83.00,12.00,0.00,0.00,0.00
Lagoon,59,"27,606,593.04",59.00,37.00,7.00,59.00,59.00,59.00


## First and latest vaults on each chain

- Show how much history we have for each chain


In [6]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level="Chain")["First seen"].agg(["min", "max"]).reset_index()

# Rename columns for clarity
seen_df.columns = ["Chain", "First vault deployed", "Last vault deployed"]

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-08-08 12:42:43
Avalanche,2022-04-11 00:05:24,2025-07-29 13:10:10
Base,2023-08-04 15:36:07,2025-08-11 05:34:11
Berachain,2025-01-26 00:06:03,2025-08-06 22:42:24
Binance,2022-05-27 17:25:18,2025-08-06 08:55:28
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-05-05 13:09:47
Ethereum,2019-06-11 06:17:19,2025-08-13 01:23:35
Gnosis,2022-05-01 18:47:35,2025-06-26 13:54:40


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [7]:
largest_threshold = 30
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100]
largest_df = largest_df[largest_df["Stablecoin denominated"] == True]
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold).reset_index())

,Name,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
0,Staked USDe,"5,426,740,789.77",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,85970,<unknown>,<unknown>
1,Savings USDS,"1,924,849,714.98",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,61299,<unknown>,<unknown>
2,Syrup USDC,"865,821,297.63",Ethereum,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,USDC,12550,<unknown>,<unknown>
3,Ethereal Pre-deposit Vault,"784,789,268.93",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,93024,<unknown>,<unknown>
4,Spark USDC Vault,"577,545,304.40",Base,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,USDC,40463,0.00,0.00
5,Savings Dai,"443,269,200.87",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,79276,<unknown>,<unknown>
6,Staked Falcon USD,"382,597,956.34",Ethereum,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,USDf,14876,<unknown>,<unknown>
7,Spark USDC Vault,"377,772,760.13",Ethereum,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,USDC,6835,<unknown>,<unknown>
8,Compounding Open Dollar,"270,227,282.16",Ethereum,0xad55aebc9b8c03fc43cd9f62260391c13c23e7c0,USDO,2615,<unknown>,<unknown>
9,Steakhouse USDC,"265,559,660.43",Ethereum,0xbeef01735c132ada46aa9aa4c54623caa92a64cb,USDC,9806,0.00,0.00


## Largest USD vault per chain

- Get the largest vault of each chain

In [8]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby("Chain")["NAV"].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain     Name                                                                
Arbitrum  ostiumLP                                            39,017,680.70   
Avalanche Staked avUSD                                        94,816,970.27   
Base      Spark USDC Vault                                   577,545,304.40   
          Spark USDC Vault                                     9,913,503.89   
Berachain Concrete USDe MultiSig Strategy                     13,441,206.33   
          Concrete USDe MultiSig Strategy                      2,292,517.72   
Binance   Staked USDX                                        225,263,682.90   
Blast     Wasabi USDB Vault                                      371,422.34   
Celo      Staked EURA                                          3,221,591.41   
Ethereum  Staked USDe                                      5,426,740,789.77   
Gnosis    Savings xDAI                                       109,877,625.31   
Mode      USDC Ironclad Vault                                    337,647.27   
Optimism  Spark USDC Vault                                    12,730,234.77   
Polygon   Compound USDC                                       26,271,577.80   
Soneium   UB-USDC                                                177,105.03   
Sonic     Silo Finance Borrowable USDC Deposit, SiloId: 20     9,502,289.13   
ZKsync    hsUSDC                                                 593,485.27   

                                                                                               Address  \
Chain     Name                                                                                           
Arbitrum  ostiumLP                                          0x20d419a8e12c45f88fda7c5760bb6923cee27f98   
Avalanche Staked avUSD                                      0x06d47f3fb376649c3a9dafe069b3d6e35572219e   
Base      Spark USDC Vault                                  0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
          Spark USDC Vault                                  0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain Concrete USDe MultiSig Strategy                   0x4cf8427e0e640ea0afff1aa0098d28200b0f4fd4   
          Concrete USDe MultiSig Strategy                   0x4ab370bb847fcc1d742d792f1bf2aff458af7c61   
Binance   Staked USDX                                       0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Blast     Wasabi USDB Vault                                 0x4bed2a922654cacc2be974689619768fabf24855   
Celo      Staked EURA                                       0x004626a008b1acdc4c74ab51644093b155e59a23   
Ethereum  Staked USDe                                       0x9d39a5de30e57443bff2a8307a4256c8797a3497   
Gnosis    Savings xDAI                                      0xaf204776c7245bf4147c2612bf6e5972ee483701   
Mode      USDC Ironclad Vault                               0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Optimism  Spark USDC Vault                                  0xcf9326e24ebffbef22ce1050007a43a3c0b6db55   
Polygon   Compound USDC                                     0x781fb7f6d845e3be129289833b04d43aa8558c42   
Soneium   UB-USDC                                           0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41   
Sonic     Silo Finance Borrowable USDC Deposit, SiloId: 20  0x322e1d5384aa4ed66aeca770b95686271de61dc3   
ZKsync    hsUSDC                                            0xaf08a9d918f16332f22cf8dc9abe9d9e14ddcbc2   

                                                           Denomination  \
Chain     Name                                                            
Arbitrum  ostiumLP                                                 USDC   
Avalanche Staked avUSD                                            avUSD   
Base      Spark USDC Vault                                         USDC   
          Spark USDC Vault                                         USDC   
Berachain Concrete USDe MultiSig Strategy                          USDe   
          Concrete USDe MultiS

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [9]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
Beraborrow iBGT,1361499,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"139,458.31",159 days 02:59:06.009241,705249,656250
Savings xDAI,920374,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"109,877,625.31",685 days 02:50:55.009241,478528,441846
Origami iBGT Auto-Compounder,820997,Berachain,0x69f1e971257419b1e9c405a553f252c64a29a30a,iBGT,"5,000,852.20",149 days 13:42:04.009241,640763,180234
UB-WETH,796374,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,44.54,212 days 08:11:20.009241,590503,205871
USDC yVault-A,490600,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"209,509.35",656 days 22:18:37.009241,465226,25374
USDT yVault-A,458879,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"321,434.51",614 days 22:02:05.009241,407866,51013
BathToken v1,297397,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"58,199.68",1176 days 15:42:21.009241,150209,147188
Bridged USDC (Stargate)Vault,294157,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,0.18,199 days 14:58:22.009241,129931,164226
Seamless USDC Vault,274553,Base,0x616a4e1db48e22028f6bbf20444cd3b8e3273738,USDC,"67,547,200.88",211 days 15:30:14.009241,150693,123860


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [10]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby("Chain")["Total events"].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"124,879.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"668,619.19",580 days 06:32:00.009241,121710,3169
Avalanche,Wrapped Aave Avalanche WAVAX,"216,480.00",0xd7da0de6ef4f51d6206bf2a35fcd2030f54c3f7b,WAVAX,"25,941.30",125 days 13:12:22.009241,116950,99530
Base,Seamless USDC Vault,"274,553.00",0x616a4e1db48e22028f6bbf20444cd3b8e3273738,USDC,"67,547,200.88",211 days 15:30:14.009241,150693,123860
Berachain,Beraborrow iBGT,"1,361,499.00",0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"139,458.31",159 days 02:59:06.009241,705249,656250
Binance,kUSDT,"43,619.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,553,043.71",793 days 07:55:42.009241,25477,18142
Blast,Wasabi WETH Vault,"244,022.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,452.68,530 days 18:48:48.009241,161390,82632
Celo,Wrapped mcUSD,260.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,039.70",1062 days 19:30:28.009241,180,80
Ethereum,pufETH,"170,484.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"39,654.41",559 days 19:19:50.009241,152802,17682
Gnosis,Savings xDAI,"920,374.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"109,877,625.31",685 days 02:50:55.009241,478528,441846


## Oldest vaults

- Show oldest vaults

In [11]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby("Chain")["Age"].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1173 days 23:49:19.009241,WETH,17.94,"2,401.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1217 days 22:07:03.009241,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,712 days 23:10:32.009241,WETH,0.75,"1,082.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,199 days 14:58:22.009241,USDC.e,0.18,"294,157.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1142 days 18:11:46.009241,NRCH,"1,168,273.97","1,201.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,530 days 18:48:48.009241,WETH,452.68,"244,022.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1206 days 03:38:48.009241,THOR,"79,695,782.99","20,847.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,712 days 01:22:30.009241,EURA,"389,522.92","3,759.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,476 days 20:52:19.009241,mETH,338.19,"13,471.00"


## All vaults

- List of all stablecoin vaults 
- We also use this list to show the vault which participate the stablecoin benchmark

In [12]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)

Displaying 837 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Staked USDe,"5,426,740,789.77",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,631 days 06:58:14.009241,USDe,85970
Savings USDS,"1,924,849,714.98",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,330 days 01:27:38.009241,USDS,61299
Syrup USDC,"865,821,297.63",Ethereum,<unknown ERC-4626>,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,423 days 20:33:14.009241,USDC,12550
Ethereal Pre-deposit Vault,"784,789,268.93",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,180 days 23:50:14.009241,USDe,93024
Spark USDC Vault,"577,545,304.40",Base,Morpho,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,209 days 18:33:10.009241,USDC,40463
Janus Henderson Anemoy AAA CLO Fund Token,"551,538,710.81",Ethereum,<unknown ERC-7540>,0xe9d1f733f406d4bbbdfac6d4cfcd2e13a6ee1d01,90 days 18:30:02.009241,USDC,14
Savings Dai,"443,269,200.87",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,868 days 04:39:38.009241,DAI,79276
Janus Henderson Anemoy Treasury Fund,"383,869,910.43",Ethereum,<unknown ERC-7540>,0x36036ffd9b1c6966ab23209e073c68eb9a992f50,127 days 17:52:38.009241,USDC,10
Staked Falcon USD,"382,597,956.34",Ethereum,<unknown ERC-4626>,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,181 days 18:11:26.009241,USDf,14876
